In [46]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [47]:
# load the dataset, split into input (X) and output (y) variables
dataset = np.loadtxt('pima-indians-diabetes.csv', delimiter=',')
X = dataset[:, 0:8]
y = dataset[:, 8]

In [48]:
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [49]:
X = torch.tensor(X, dtype=torch.float32)

# should be column
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)


In [50]:
ex = np.array([1, 2, 3])
ex.reshape(-1, 1)

array([[1],
       [2],
       [3]])

# first way

In [51]:
model = nn.Sequential(
    nn.Linear(8, 12),
    nn.ReLU(),
    nn.Linear(12, 8),
    nn.ReLU(),
    nn.Linear(8, 1),
    nn.Sigmoid()
)
print(model)

Sequential(
  (0): Linear(in_features=8, out_features=12, bias=True)
  (1): ReLU()
  (2): Linear(in_features=12, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=1, bias=True)
  (5): Sigmoid()
)


# better way

In [52]:
class PrimeClassifier(nn.Module):
    def __init__(self):
        # super(PrimeClassifier, self).__init__()
        super().__init__()
        self.hidden1 = nn.Linear(8, 12)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(12, 8)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(8, 1)
        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x


model = PrimeClassifier()
print(model)

PrimeClassifier(
  (hidden1): Linear(in_features=8, out_features=12, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=12, out_features=8, bias=True)
  (act2): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (act_output): Sigmoid()
)


In [53]:
loss_fun = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [54]:
n_epochs = 100
batch_size = 10

for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i + batch_size]
        y_pred = model(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fun(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Finished epoch {epoch}, latest loss {loss}")

Finished epoch 0, latest loss 0.6970937848091125
Finished epoch 1, latest loss 0.6729878783226013
Finished epoch 2, latest loss 0.6429826617240906
Finished epoch 3, latest loss 0.6255437731742859
Finished epoch 4, latest loss 0.6032555103302002
Finished epoch 5, latest loss 0.5957046747207642
Finished epoch 6, latest loss 0.5865129232406616
Finished epoch 7, latest loss 0.5794280767440796
Finished epoch 8, latest loss 0.5678671002388
Finished epoch 9, latest loss 0.5553832054138184
Finished epoch 10, latest loss 0.5499396920204163
Finished epoch 11, latest loss 0.5397990345954895
Finished epoch 12, latest loss 0.5283559560775757
Finished epoch 13, latest loss 0.5250282287597656
Finished epoch 14, latest loss 0.5136772394180298
Finished epoch 15, latest loss 0.5071898698806763
Finished epoch 16, latest loss 0.5033526420593262
Finished epoch 17, latest loss 0.49520179629325867
Finished epoch 18, latest loss 0.4963397979736328
Finished epoch 19, latest loss 0.48769259452819824
Finished ep

In [55]:
# compute accuracy (no_grad is optional)
with torch.no_grad():
    y_pred = model(X)
    
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

Accuracy 0.765625
